***
### Tutorial to load a keras model with custom layer(s):

This notebook can be used to load the model with custom layer using the function `loading_pruned_model` available in the package. The necessary imports can be installed using the cell below. For all the classes that are not buint-in to the tensorflow/keras library, you have to use the `custom_objects` parameter to specify them so that when reading the .json file to ascertain the model architecture, the function `load_model_from_json` can identify any customized layers/operations. 

The format to declare the custom class object in the function call is `"Class/Function Name String": Class/ Function`. Example: for our custom layer the declaration is as follow: `"MaskedDense": MaskedDense`.
***

In [1]:
import tensorflow as tf 
import keras
import numpy as np
import pandas as pd
print(tf.__version__)
print(keras.__version__)
print(pd.__version__)
print(np.__version__)

from nn_globals import *
from nn_evaluate import get_sparsity,k_fold_validation
from nn_pruning_module_support import __generate_delta_plots__
from dataset import muon_data_split
from nn_pruning_module_support import loading_trained_model

from custom_dense_layer import MaskedDense
from keras.regularizers import l1_l2
from keras.initializers import glorot_uniform

2.6.0
2.6.0
1.3.4
1.19.5


In [4]:
# pruned_model = loading_trained_model(filepath = "./models",
#                                  model_filename = "custom_model_50",
#                                  custom_objects ={'GlorotUniform': glorot_uniform(), 
#                                                   "MaskedDense": MaskedDense, 
#                                                   "L1L2": l1_l2()})
# pruned_model.summary()
baseline = loading_trained_model(filepath = "./models",
                                 model_filename = "model")
baseline.summary()

./models/model.json
./models/model_weights.h5
Loaded model from disk
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 23)                92        
_________________________________________________________________
dense (Dense)                (None, 20)                460       
_________________________________________________________________
batch_normalization_1 (Batch (None, 20)                80        
_________________________________________________________________
activation (Activation)      (None, 20)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 15)                300       
_________________________________________________________________
batch_normalization_2 (Batch (None, 15)                60        
_____________________________________________________

In [5]:
# verify the sparsity of loaded pruned model
for layer in pruned_model.layers:
    if "dense" in layer.name:
        print("Sparsity of the {} layer is = {}".format(layer.name, get_sparsity(layer.get_weights()[0])))

Sparsity of the dense layer is = 0.0
Sparsity of the dense_1 layer is = 0.0
Sparsity of the dense_2 layer is = 0.0
Sparsity of the dense_3 layer is = 0.0


In [7]:
def expectation(array):
    n = len(array)
    prb = 1 / n
    # calculating expectation overall
    exp = 0
    for i in range(0, n):
        exp += (array[i] * prb)         
    # returning expectation as sum
    return exp

In [8]:
def weights_adjusted(W_sparse, W_dense):
    eps = 10e-9
    const = np.std(W_sparse)/(np.std(W_dense)+eps)
    W_sparse_corr = const*W_sparse + expectation(W_dense) - expectation(const*W_sparse)
    
    return W_sparse_corr
    

In [ ]:
for layer in pruned_model.layers:
    if "dense" in layer.name:
        print("Sparsity of the {} layer is = {}".format(layer.name, get_sparsity(layer.get_weights()[0])))

***
### Evaluating the model:
To use the model for prediction, suggestion is to use the functions available in `nn_evaluate` file. The code to preprocess the data can be used to load and extract relevant features from the raw file. **NOTE:** file path to the data needs to be edited in the `nn_globals.py`
***

In [ ]:
x,y,dxy =  muon_data_split(infile_muon_displ, 
                            reg_pt_scale=reg_pt_scale, 
                            reg_dxy_scale=reg_dxy_scale, 
                            test_size=0.0)
y = np.abs(y)

***
With `k_fold_validation`, using `metric_type` argument, we can change the type of evaluation metric. It can be `["RMSE", "MAE", "MAPE"]`

Using `__generate_delta_plots__` we can get a gaussian approximation for the errors for momentum and angular displacement.
***

In [ ]:
k_fold_validation(model = pruned_model,
                    x = x,
                    y = y,
                    dxy = dxy,
                    folds=1,
                    eval_batch_size=2000,
                    metric_type='RMSE')

In [ ]:
__generate_delta_plots__(model = pruned_model,
                        x = x,
                        y = y,
                        dxy = dxy,
                        color='plum',
                        bins_y = [-20.,20.],
                        bins_dxy = [-20.,20.],
                        batch_size = 4096,
                        min_y_val= 20.0)